In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

driver.get("https://results.nyrr.org/event/M2024/finishers")
time.sleep(5)

# Cargar hasta que tengamos al menos 10 atletas visibles
while True:
    atletas = driver.find_elements(By.CSS_SELECTOR, "div[ng-repeat='eventFinisher in eventFinishers']")
    if len(atletas) >= 10:
        break
    try:
        driver.find_element(By.CSS_SELECTOR, "button[ng-click='loadMore()']").click()
        time.sleep(2)
    except:
        break

resultados = []
for atleta in atletas[:10]:
    try:
        nombre = atleta.find_element(By.CSS_SELECTOR, "div.name").text
    except:
        nombre = "Sin nombre"

    try:
        edad = atleta.find_element(By.CSS_SELECTOR, "span[ng-if='eventFinisher.gender || eventFinisher.age']").text
    except:
        edad = "Sin edad"

    try:
        pais = atleta.find_element(By.CSS_SELECTOR, "span[ng-if='eventFinisher.iaaf']").text
    except:
        pais = "Sin país"

    # Extraer "Place" y "Time" correctamente
    spans_result = atleta.find_elements(By.CSS_SELECTOR, "span.result")
    posicion = "Sin posición"
    tiempo = "Sin tiempo"

    for span in spans_result:
        try:
            texto = span.text.strip()
            if "Place" in texto:
                posicion = span.find_element(By.CSS_SELECTOR, ".num.ng-binding").text
            elif "Time" in texto:
                tiempo = span.find_element(By.CSS_SELECTOR, ".num.ng-binding").text
        except:
            continue

    resultados.append({
        "nombre": nombre,
        "pais": pais,
        "edad": edad,
        "posicion": posicion,
        "tiempo": tiempo
    })

driver.quit()

df = pd.DataFrame(resultados)
print("\n✅ Prueba de 10 atletas corregida. Guardado en: 'nyc_marathon_2024_test_10.csv'")


✅ Prueba de 10 atletas corregida. Guardado en: 'nyc_marathon_2024_test_10.csv'


In [6]:
df

,nombre,pais,edad,posicion,tiempo
0,Abdi Nageeye,NED,M35,1,2:07:39
1,Evans Chebet,KEN,M35,2,2:07:45
2,Albert Korir,KEN,M30,3,2:08:00
3,Tamirat Tola,ETH,M33,4,2:08:12
4,Geoffrey Kamworor,KEN,M31,5,2:08:50
5,Conner Mantz,USA,M27,6,2:09:00
6,Clayton Young,USA,M31,7,2:09:21
7,Abel Kipchumba,KEN,M30,8,2:10:39
8,Bashir Abdi,BEL,M35,9,2:10:39
9,Cj Albertson,USA,M31,10,2:10:57


### SCRAPEO FINAL

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
from tqdm import tqdm

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
driver = webdriver.Chrome(options=options)

base_url = "https://results.nyrr.org/event/M2024/finishers#page="
pagina = 1
todos_resultados = []
rank = 1
limite_paginas = 1000  # Por seguridad para evitar bucles infinitos

print("\n📥 Iniciando scrapeo con cambio de página (#page=N)...\n")

while pagina <= limite_paginas:
    print(f"🔄 Cargando página {pagina}...")
    driver.get(f"{base_url}{pagina}")
    time.sleep(4)

    atletas = driver.find_elements(By.CSS_SELECTOR, "div[ng-repeat='eventFinisher in eventFinishers']")
    if not atletas:
        print("✅ Fin del listado: no hay más atletas.")
        break

    for atleta in tqdm(atletas, desc=f"Extrayendo atletas página {pagina}", ncols=100):
        try:
            nombre = atleta.find_element(By.CSS_SELECTOR, "div.name").text
        except:
            nombre = "Sin nombre"

        try:
            edad = atleta.find_element(By.CSS_SELECTOR, "span[ng-if='eventFinisher.gender || eventFinisher.age']").text
        except:
            edad = "Sin edad"

        try:
            pais = atleta.find_element(By.CSS_SELECTOR, "span[ng-if='eventFinisher.iaaf']").text
        except:
            pais = "Sin país"

        # Tiempo y posición desde bloques 'result'
        spans = atleta.find_elements(By.CSS_SELECTOR, "span.result")
        posicion = "Sin posición"
        tiempo = "Sin tiempo"
        for span in spans:
            try:
                if "Place" in span.text:
                    posicion = span.find_element(By.CLASS_NAME, "num").text
                elif "Time" in span.text:
                    tiempo = span.find_element(By.CLASS_NAME, "num").text
            except:
                continue

        todos_resultados.append({
            "rank": rank,
            "nombre": nombre,
            "pais": pais,
            "edad": edad,
            "posicion": posicion,
            "tiempo": tiempo
        })

        rank += 1

    pagina += 1

driver.quit()

# Guardar
df_NYC = pd.DataFrame(todos_resultados)
df_NYC.to_csv("../data/resultados_nyc_marathon_2024_completo_paginas.csv", index=False)
print(f"\n✅ Scrapeo completado con {len(df)} atletas.")
print("📁 Archivo guardado: nyc_marathon_2024_completo_paginas.csv")


📥 Iniciando scrapeo con cambio de página (#page=N)...

🔄 Cargando página 1...


Extrayendo atletas página 1: 100%|██████████████████████████████████| 51/51 [00:01<00:00, 31.56it/s]


🔄 Cargando página 2...


Extrayendo atletas página 2: 100%|██████████████████████████████████| 51/51 [00:01<00:00, 31.00it/s]


🔄 Cargando página 3...


...

🔄 Cargando página 55...


Extrayendo atletas página 55: 100%|███████████████████████████████| 102/102 [00:03<00:00, 26.58it/s]


🔄 Cargando página 56...


Extrayendo atletas página 56:   9%|██▉                              | 9/102 [00:00<00:04, 22.96it/s]


MaxRetryError: HTTPConnectionPool(host='localhost', port=61538): Max retries exceeded with url: /session/6dff75126053b55bbb65124096e2407c/element/f.723FF5DF626A22D4D6293FF19404007C.d.BFA26E8AC5482FA0DEC00D925AF68880.e.1399/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10ec910a0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [10]:
df_NYC

,nombre,pais,edad,posicion,tiempo
0,Abdi Nageeye,NED,M35,Time 2:07:39,Sin tiempo
1,Evans Chebet,KEN,M35,Time 2:07:45,Sin tiempo
2,Albert Korir,KEN,M30,Time 2:08:00,Sin tiempo
3,Tamirat Tola,ETH,M33,Time 2:08:12,Sin tiempo
4,Geoffrey Kamworor,KEN,M31,Time 2:08:50,Sin tiempo
5,Conner Mantz,USA,M27,Time 2:09:00,Sin tiempo
6,Clayton Young,USA,M31,Time 2:09:21,Sin tiempo
7,Abel Kipchumba,KEN,M30,Time 2:10:39,Sin tiempo
8,Bashir Abdi,BEL,M35,Time 2:10:39,Sin tiempo
9,Cj Albertson,USA,M31,Time 2:10:57,Sin tiempo
